%%latex
\tableofcontents

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
import random
import csv
import h5py
import pandas as pd
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.cm as cm

# Export the NNC2P model

This is the model architecture:

In [2]:
# Define hyperparameters of the model here. Will first of all put two hidden layers
# total of 800 neurons for the one in the paper
device = "cpu"
size_HL_1 = 600
size_HL_2 = 200

# Implement neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(3, size_HL_1),
            nn.Sigmoid(),
            nn.Linear(size_HL_1, size_HL_2),
            nn.Sigmoid(),
            nn.Linear(size_HL_2, 1)
        )

    def forward(self, x):
        # No flatten needed, as our input and output are 1D?
        #x = self.flatten(x) 
        logits = self.stack(x)
        return logits

We import NNC2Pv0, which was on par with the models in the paper.

In [3]:
NNC2P = torch.load('Models/NNC2Pv0.pth')
model = NNC2P

Look at the parameter values:

In [4]:
with torch.no_grad():
    for param in NNC2P.parameters():
        print(param)

Parameter containing:
tensor([[-0.3637,  0.4540, -0.4355],
        [ 0.0066,  0.6949,  0.4879],
        [ 0.1112, -0.0925,  0.1091],
        ...,
        [ 0.5306, -0.4535, -0.3026],
        [-0.4308, -0.1415,  0.2810],
        [ 0.6349, -0.2947,  0.0561]], requires_grad=True)
Parameter containing:
tensor([ 0.5675,  0.2904, -0.7667, -0.3078, -0.1945,  0.0523,  0.0514, -0.4138,
         0.2312, -0.5222,  0.2495, -0.3197, -0.4844, -0.5024, -0.3668, -0.2699,
         0.7860,  0.7489,  0.1024,  0.8798,  0.1536, -0.4353, -0.3389, -0.5969,
        -0.4334, -0.7355, -0.4756, -0.4140, -0.1220, -0.1788, -0.7250, -0.0075,
         0.2842,  0.1193,  0.5405, -0.1805, -0.0228, -0.3408, -0.1134, -0.2822,
         0.5498, -0.1406,  0.3311, -0.5858,  0.0567, -0.2661,  0.3879,  0.8417,
        -0.2426,  0.5311,  0.0035,  0.1361, -0.3355,  0.2191, -0.3657,  0.0739,
        -0.7668, -0.7611, -0.4528,  0.7155,  0.4711,  0.1546, -0.7966, -0.6006,
         0.5338, -0.4438, -0.5507,  0.2647, -0.5531, -0.1843

We follow [this guide from the PyTorch documentation](https://pytorch.org/tutorials/advanced/cpp_export.html). 

## Get the parameters as matrices

In [5]:
# torch.save(NNC2P.state_dict(), "NNC2Pv0_params.h5")

In [6]:
NNC2P.state_dict()

OrderedDict([('stack.0.weight',
              tensor([[-0.3637,  0.4540, -0.4355],
                      [ 0.0066,  0.6949,  0.4879],
                      [ 0.1112, -0.0925,  0.1091],
                      ...,
                      [ 0.5306, -0.4535, -0.3026],
                      [-0.4308, -0.1415,  0.2810],
                      [ 0.6349, -0.2947,  0.0561]])),
             ('stack.0.bias',
              tensor([ 0.5675,  0.2904, -0.7667, -0.3078, -0.1945,  0.0523,  0.0514, -0.4138,
                       0.2312, -0.5222,  0.2495, -0.3197, -0.4844, -0.5024, -0.3668, -0.2699,
                       0.7860,  0.7489,  0.1024,  0.8798,  0.1536, -0.4353, -0.3389, -0.5969,
                      -0.4334, -0.7355, -0.4756, -0.4140, -0.1220, -0.1788, -0.7250, -0.0075,
                       0.2842,  0.1193,  0.5405, -0.1805, -0.0228, -0.3408, -0.1134, -0.2822,
                       0.5498, -0.1406,  0.3311, -0.5858,  0.0567, -0.2661,  0.3879,  0.8417,
                      -0.2426,  0.5311

### First, make it easy: save the matrices as a CSV

In [7]:
param_names = NNC2P.state_dict().keys()
file_names = ["Models/NNC2Pv0_params_" + key + ".csv" for key in param_names]


for i, key in enumerate(param_names):
    # Get the key and name of the current parameter matrix that we are saving
    # key = param_names[i]
    name = file_names[i]
    print(name)
    # Get the value of these parameters:
    matrix = NNC2P.state_dict()[key]
    # Convert to Numpy array
    matrix_np = matrix.numpy() 
    # Convert to a dataframe
    df = pd.DataFrame(matrix_np)
    # Save to file
    df.to_csv(name,index=False, header=False)

Models/NNC2Pv0_params_stack.0.weight.csv
Models/NNC2Pv0_params_stack.0.bias.csv
Models/NNC2Pv0_params_stack.2.weight.csv
Models/NNC2Pv0_params_stack.2.bias.csv
Models/NNC2Pv0_params_stack.4.weight.csv
Models/NNC2Pv0_params_stack.4.bias.csv


Read the files:

In [8]:
weight0 = pd.read_csv("Models/NNC2Pv0_params_stack.0.weight.csv", header=None).values
bias0   = pd.read_csv("Models/NNC2Pv0_params_stack.0.bias.csv", header=None).values
weight2 = pd.read_csv("Models/NNC2Pv0_params_stack.2.weight.csv", header=None).values
bias2   = pd.read_csv("Models/NNC2Pv0_params_stack.2.bias.csv", header=None).values
weight4 = pd.read_csv("Models/NNC2Pv0_params_stack.4.weight.csv", header=None).values
bias4   = pd.read_csv("Models/NNC2Pv0_params_stack.4.bias.csv", header=None).values

weights_and_biases = [weight0, bias0, weight2, bias2, weight4, bias4]

In [9]:
# Print the shape for each parameter:
for i in range(len(weights_and_biases)):
    print("For the file: ", file_names[i])
    # Read the values
    shape = np.shape(weights_and_biases[i])
    print("The shape is equal to ", shape)

For the file:  Models/NNC2Pv0_params_stack.0.weight.csv
The shape is equal to  (600, 3)
For the file:  Models/NNC2Pv0_params_stack.0.bias.csv
The shape is equal to  (600, 1)
For the file:  Models/NNC2Pv0_params_stack.2.weight.csv
The shape is equal to  (200, 600)
For the file:  Models/NNC2Pv0_params_stack.2.bias.csv
The shape is equal to  (200, 1)
For the file:  Models/NNC2Pv0_params_stack.4.weight.csv
The shape is equal to  (1, 200)
For the file:  Models/NNC2Pv0_params_stack.4.bias.csv
The shape is equal to  (1, 1)


In [10]:
# Read the example file
example = pd.read_csv("Models/NNC2Pv0_params_stack.0.weight.csv", header=None).values
print(example)
print(np.shape(example))

[[-0.36373898  0.45402282 -0.4355268 ]
 [ 0.00657238  0.69492054  0.4879491 ]
 [ 0.11119203 -0.09253014  0.10905483]
 ...
 [ 0.5305801  -0.45353398 -0.3026449 ]
 [-0.4308225  -0.14152803  0.28101048]
 [ 0.63488334 -0.29469463  0.05608372]]
(600, 3)


### Now, also export their 'flattened versions', as we are struggling with arrays in Fortran

In [24]:
weight0_flat = weight0.flatten()
bias0_flat      = bias0.flatten()
weight2_flat = weight2.flatten()
bias2_flat      = bias2.flatten()
weight4_flat = weight4.flatten()
bias4_flat      = bias4.flatten()

In [25]:
print(weight0_flat)

[-0.36373898  0.45402282 -0.4355268  ...  0.63488334 -0.29469463
  0.05608372]


In [28]:
head_bool = True
# Now to csv files:
df = pd.DataFrame(weight0_flat)
df.to_csv("weight0_flat.csv", index=False, header=head_bool)
df = pd.DataFrame(bias0_flat)
df.to_csv("bias0_flat.csv",      index=False, header=head_bool)
df = pd.DataFrame(weight2_flat)
df.to_csv("weight2_flat.csv", index=False, header=head_bool)
df = pd.DataFrame(bias2_flat)
df.to_csv("bias2_flat.csv",      index=False, header=head_bool)
df = pd.DataFrame(weight4_flat)
df.to_csv("weight4_flat.csv", index=False, header=head_bool)
df = pd.DataFrame(bias4_flat)
df.to_csv("bias4_flat.csv",      index=False, header=head_bool)

### How to use the parameters to make a prediction without any Pytorch tools

When we are going to implement this in the Gmunu code, we can no longer use any of the built-in tools of PyTorch.

In [53]:
## One specific test case for the data
rho,eps,v,p,D,S,tau = 9.83632270803203,1.962038705851822,0.2660655147967911,12.866163917605371,10.204131145455385,12.026584842282125,22.131296926293793

This is how the PyTorch implementation works:

In [54]:
input_test = torch.tensor([D, S, tau])
exact_result = p
print(exact_result)
print(input_test)
with torch.no_grad():
    pred = model(input_test).item()
print(pred)

12.866163917605371
tensor([10.2041, 12.0266, 22.1313])
12.86711311340332


Now, we have to try and get the same output, but by defining all intermediate steps ourselves!

In [76]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def compute_prediction(x):
    """Input is a np. array of size 1x3"""
    print(np.shape(x))
    x = np.matmul(weight0, x.T) + bias0
    print(np.shape(x))
    x = sigmoid(x)
    x = np.matmul(weight2, x) + bias2
    print(np.shape(x))
    x = sigmoid(x)
    x = np.matmul(weight4, x) + bias4
    print(np.shape(x))
    return x[0][0]

In [77]:
input_test = np.array([[D, S, tau]])
print(np.shape(input_test))

(1, 3)


In [78]:
our_prediction = compute_prediction(input_test)
print(our_prediction)
print(pred)

(1, 3)
(600, 1)
(200, 1)
(1, 1)
12.867113930614748
12.86711311340332


Now we compute rho and eps from this (see appendix A of central paper)

In [81]:
v_star = S/(tau + D + our_prediction)
W_star = 1/np.sqrt(1-v_star**2)

rho_star = D/W_star
eps_star = (tau + D*(1 - W_star) + our_prediction*(1 - W_star**2))/(D*W_star)
print("Our calculations:")
print(rho_star, eps_star)
print("Exact results:")
print(rho, eps)

Our calculations:
9.836338457223192 1.9620408002397969
Exact results:
9.83632270803203 1.962038705851822


### Now save it into an hdf5 file

In [9]:
# # Open an HDF5 file for writing
# with h5py.File("NNC2Pv0_params.h5", "w") as f:
#     # Save the weights and biases of the network to the HDF5 file
#     f.create_dataset("NNC2Pv0_params", data=NNC2P.state_dict())

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

## More advanced: using Torch script

There exist two ways of converting a PyTorch model to Torch Script. The first is known as tracing, a mechanism in which the structure of the model is captured by evaluating it once using example inputs, and recording the flow of those inputs through the model. This is suitable for models that make limited use of control flow. The second approach is to add explicit annotations to your model that inform the Torch Script compiler that it may directly parse and compile your model code, subject to the constraints imposed by the Torch Script language.

In [10]:
example = torch.tensor([1, 1, 0.5])
example

tensor([1.0000, 1.0000, 0.5000])

In [12]:
traced_script_module = torch.jit.trace(model, example)
traced_script_module

NeuralNetwork(
  original_name=NeuralNetwork
  (stack): Sequential(
    original_name=Sequential
    (0): Linear(original_name=Linear)
    (1): Sigmoid(original_name=Sigmoid)
    (2): Linear(original_name=Linear)
    (3): Sigmoid(original_name=Sigmoid)
    (4): Linear(original_name=Linear)
  )
)

In [16]:
output = traced_script_module(torch.tensor([1,1,0.5]))
output

tensor([0.0595], grad_fn=<AddBackward0>)

In [17]:
traced_script_module.save("traced_NNC2P_model.pt")

__To do: finish it__